In [3]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")
# ---------------------------------------------
# 1) Buscar BOMs que tengan routing
# ---------------------------------------------
bom_ids = models.execute_kw(
    db, uid, password,
    'mrp.bom', 'search',
    [[('routing_id', '!=', False)]],  # <= Solo BOM con Routing
)

print(f"BOMs con routing encontradas: {len(bom_ids)}")

if not bom_ids:
    print("No se encontraron BOM con routing.")
    raise SystemExit()

# Leer solo product_tmpl_id de las BOM
boms = models.execute_kw(
    db, uid, password,
    'mrp.bom', 'read',
    [bom_ids],
    {'fields': ['id', 'product_tmpl_id', 'routing_id']}
)

product_template_ids = sorted({b['product_tmpl_id'][0] for b in boms})
print(f"Templates con routing encontrados: {len(product_template_ids)}")

# ---------------------------------------------
# 2) Obtener product.product a partir del template
# ---------------------------------------------
product_ids = models.execute_kw(
    db, uid, password,
    'product.product', 'search',
    [[('product_tmpl_id', 'in', product_template_ids)]],
)


BOMs con routing encontradas: 739
Templates con routing encontrados: 738
